In [44]:
from pathlib import Path

MAX_DATA_PATH = Path("C:/Users/maxpd/Desktop/483 ML/Intrusion Detection/483-final-project-datasets/OPCUA_dataset_public.csv")
CHRIS_DATA_PATH = Path("/Users/chriszinati/Desktop/CPSC-483/final project datasets/483-final-project-datasets/OPCUA_dataset_public.csv")
MAX_DATA_PATH_REDUCED_CLEANED = Path("C:/Users/maxpd/Desktop/483 ML/Intrusion Detection/483-final-project-datasets/cleaned_data_for_OCPUA.csv")
CHRIS_DATA_PATH_CLEANED = Path("/Users/chriszinati/Desktop/CPSC-483/final project datasets/483-final-project-datasets/cleaned_data_for_OCPUA.csv")



**build 2 models of same algorithm using full and reduced feature set**
we are using random forest here.
1. import all necessary libraries

In [45]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

2. create full feature and reduced datasets from csv (we already reduced features during preprocessing, so we will bring the original dataset and clean it but without dropping features (only dropping instances that aren't DDoS or regular traffic))

In [46]:
full_features = pd.read_csv(CHRIS_DATA_PATH)
preprocessed_reduced = pd.read_csv(CHRIS_DATA_PATH_CLEANED)

full_features = full_features.dropna(axis=0) #drop rows with missing values
instancesToDrop= ['MITM', 'Impersonation'] 
full_features = (full_features[full_features.multi_label.isin(instancesToDrop) == False]) #drop instances of MITM and Impersonation
full_features = full_features.drop('multi_label', axis=1) #drop multiclass label

from sklearn.preprocessing import LabelEncoder

for col in ['proto', 'service', 'service_errors', 'status_errors', 'src_ip', 'dst_ip']:
    if full_features[col].dtype == 'object' or full_features[col].dtype.name == 'category':
        le = LabelEncoder()
        full_features[col] = le.fit_transform(full_features[col])


X_full = full_features.iloc[:, :-1]
y_full = full_features.iloc[:, -1]

X_reduced = preprocessed_reduced.iloc[:, :-1]
y_reduced = preprocessed_reduced.iloc[:, -1]


3. train 2 random forest models on reduced and full-feature training sets

In [47]:
X_reduced_train, X_reduced_test, y_reduced_train, y_reduced_test = train_test_split(X_reduced, y_reduced, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = RandomForestClassifier(n_estimators=100, random_state=42)

model1.fit(X_reduced_train, y_reduced_train)
model2.fit(X_train, y_train)

filename='finalized_model_M1.sav'
filename2='finalized_model_M2.sav'

pickle.dump(model1, open(filename, 'wb'))
pickle.dump(model2, open(filename2, 'wb'))

**load the saved models from disc and test**

In [48]:
loaded_model_1 = pickle.load(open(filename, 'rb'))
loaded_model_2 = pickle.load(open(filename2, 'rb'))

In [49]:
result = loaded_model_1.score(X_reduced_test, y_reduced_test)
result2 = loaded_model_2.score(X_test, y_test)

print(result)
print(result2)

1.0
1.0


In [50]:
import os
print(os.getcwd())

/Users/chriszinati
